In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
%matplotlib widget

In [18]:
import pandas as pd
import os
from pathlib import Path
import wandb

os.chdir('/home/bohdan/studies/sem8/io')
# os.chdir('/home/ehevi/studies/io-aco/')

In [19]:
csv_path = Path('small.csv')
problem = pd.read_csv(csv_path)
problem.head()


,cust,x,y,demand,from,to,service
0,1,10.0,50.0,0.0,0.0,1236.0,0.0
1,2,15.0,68.0,10.0,912.0,967.0,90.0
2,3,15.0,70.0,30.0,825.0,870.0,90.0
3,4,12.0,66.0,10.0,65.0,146.0,90.0
4,5,12.0,68.0,10.0,727.0,782.0,90.0


1. Uruchomić
 - dostosowanie do naszych danych
 
2. Dodać modyfikacje
- dodajemy do mrówki wiedzę o depocie oraz o pojemności
- mrówka po osiągnięciu maks pojemności wraca do początku



In [20]:
import random
from swarmlib_.aco4tsp.aco_algorithm import ACOAlgorithm
from swarmlib_.aco4tsp.vehicle import Vehicle
import numpy as np
from itertools import product

def evaluate_scenario(path: Path):
    vehicles = [Vehicle(50, 1) for _ in range(10)]
    config = {
        'ant_number': 1,
        'num_vehicles': 10,
        'iteration_number': 350,
        'capacity': 50,
        'commit_freq': 10,
        'vehicles': vehicles,
        'instance_path': 'small.csv',
        'rho': 0.5,
        'alpha': 0.5,
        'beta': 0.5,
    }

    
    random.seed(42)
    for alpha, rho, beta in product(np.arange(0.1, 1, 0.3), np.arange(0.1, 1, 0.3), np.arange(0.1, 1, 0.3)):
        wandb.init(
            project="dynamic aco",
            config={
                **config,
                'rho': rho,
                'alpha': alpha,
                'beta': beta,
            },
            tags=[f'07-06-2023-{path.stem}-fixed-aco-fixed-scenario']
        )
        # Logic goes here
        try:
            solver = ACOAlgorithm(**config, scenario_input_file=path)
            best_plan, _ = solver.solve()
        finally:
            wandb.finish()


In [21]:
for scenario in ['scenario_mix.csv']:
    evaluate_scenario(Path(scenario))